In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.contrib.layers as tfl

/Users/simon/Programs/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
X_tf = tf.placeholder(shape=[None, 2], dtype=tf.float32)
Y_tf = tf.placeholder(shape=[None, 1], dtype=tf.float32) #There should be just one result

net = tf.identity(X_tf) # Is just net=X_tf
#net<- sigma(A*net+a)

# http://colah.github.io/posts/2014-03-NN-Manifolds-Topology/
net = tfl.fully_connected(net,  2, activation_fn=tf.nn.sigmoid) # <==== this layer is important activation function is something non linear
#If we add another element to the layer 2-> 3 the accuracy is 100%
#Why do we need an activation function? is to break the holomorfism
net = tfl.fully_connected(net,  1, activation_fn=tf.identity) 

loss_tf = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y_tf, logits=net)) 

# sigmoid_cross_entropy_with_logits is a function that takes care that the logaritm does not...


step_tf = tf.train.AdamOptimizer(1E-1).minimize(loss_tf) #AdamOptimizer is gradient descent on steroids

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    num_iterations, print_every, batch_size = 2**12, 2**10, 2**10
    for iteration in range(num_iterations):
        
        X = np.random.randint(0, 2, (batch_size, 2))
        Y = (X[:,:1]+X[:,1:]) % 2
         
        sess.run(step_tf, feed_dict={X_tf: X, Y_tf: Y})
        
        if iteration % print_every == print_every-1:
            loss = sess.run(loss_tf, feed_dict={X_tf: X, Y_tf: Y})
            print("loss", loss)

    #These are the real values
    X = np.random.randint(0, 2, (batch_size, 2))
    Y = (X[:,:1]+X[:,1:]) % 2
    
    Y_ = sess.run(tf.nn.sigmoid(net), feed_dict={X_tf: X, Y_tf: Y})
    print("accuracy:", np.mean(np.round(Y_).astype(np.int8) == Y))

('loss', 0.34854388)
('loss', 0.35040617)
('loss', 0.33949804)
('loss', 0.33983636)
('accuracy:', 0.7431640625)


# A fully connected layer is a continuos function, read 
http://colah.github.io/posts/2014-03-NN-Manifolds-Topology/

If the topology is continuos in 2D, then you can use 3 dimensions, then it will identify the domains, see blog for the two concentrical rings.